# Deep-Learning Classification of Play Type in NFL Play-By-Play Data

### Ian Johnson, Derek Phanekham, Travis Siems

## Introduction

The NFL (National Football League) has 32 teams split into two conferences, the AFC and NFC. Each of the 32 teams plays 16 games during the regular season (non-playoff season) every year. Due to the considerable viewership of American football, as well as the pervasiveness of fantasy football, considerable data about the game is collected. During the 2015-2016 season, information about every play from each game that occurred was logged. All of that data was consolidated into a single data set which is analyzed throughout this report.

In this report, we will attempt to classify the type of a play, given the game situation before the play began. 

### The Classification Task

We will attempt to classify plays based on play type using information about the state of the game prior to the start of the play. This is expected to be an exceptionally difficult classification task, due to the amount of noise in the dataset (specifically, the decision to run vs pass the ball is often a seemingly random one). A successful classifier would have huge value to defensive coordinators, who could call plays based on the expected offensive playcall. Because it may be very difficult to identify what play will be called, it is relevant to provide a probability of a given playcall in a situation. For example, it would be useful to provide the probability of a 4th down conversion attempt, even if the overall prediction is that a punt occurs. 

### Data Preparation

In order to prepare the data for classification, a number of variables from the original dataset will be removed, as they measure the result of the play, not the state of the game prior to the start of the play. The dataset being included in this report has had previous cleaning and preprocessing performed in our previous report.

In [2]:
#For final version of report, remove warnings for aesthetics.
import warnings
warnings.filterwarnings('ignore')

#Libraries used for data analysis
import pandas as pd
import numpy as np
from sklearn import preprocessing

df = pd.read_csv('data/cleaned.csv') # read in the csv file



colsToInclude = [ 'Drive', 'qtr', 'down',
                 'TimeSecs', 'yrdline100','ydstogo','ydsnet',
                 'GoalToGo','posteam','DefensiveTeam',
                 'PosTeamScore','ScoreDiff', 'PlayType']

df = df[colsToInclude]
df = df[[p not in ["Sack", "No Play", "QB Kneel", "Spike"] for p in df.PlayType]]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38600 entries, 0 to 42875
Data columns (total 13 columns):
Drive            38600 non-null int64
qtr              38600 non-null int64
down             38600 non-null int64
TimeSecs         38600 non-null float64
yrdline100       38600 non-null float64
ydstogo          38600 non-null float64
ydsnet           38600 non-null float64
GoalToGo         38600 non-null int64
posteam          38600 non-null object
DefensiveTeam    38600 non-null object
PosTeamScore     38600 non-null float64
ScoreDiff        38600 non-null float64
PlayType         38600 non-null object
dtypes: float64(6), int64(4), object(3)
memory usage: 4.1+ MB


#### Neural Network Embeddings

We will use neural network embeddings from TensorFlow for the posteam and DefensiveTeam. However, we will be building these embeddings manually using one-hot encoding and additional fully-connected layers in each of the deep architectures. The following Python function was used for one-hot encoding, and was adapted from the website referenced in the code.

In [3]:
from sklearn.feature_extraction import DictVectorizer

#Simple function for 1 hot encoding
def encode_onehot(df, cols):
    """
    One-hot encoding is applied to columns specified in a pandas DataFrame.
    
    Modified from: https://gist.github.com/kljensen/5452382
    
    Details:
    
    http://en.wikipedia.org/wiki/One-hot
    http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html
    
    @param df pandas DataFrame
    @param cols a list of columns to encode
    @return a DataFrame with one-hot encoding
    """
    vec = DictVectorizer()
    
    vec_data = pd.DataFrame(vec.fit_transform(df[cols].to_dict(outtype='records')).toarray())
    vec_data.columns = vec.get_feature_names()
    vec_data.index = df.index
    
    df = df.drop(cols, axis=1)
    df = df.join(vec_data)
    return df

df = encode_onehot(df, cols=['posteam', 'DefensiveTeam'])

The following are descriptions of the remaining data columns in the play-by-play dataset. Note that the one-hot encoded columns do not follow the structure listed below, but for the sake of readability they are presented as if they were not one-hot encoded.

* **GameID** (*nominal*): A unique integer which identifies each game played 
* **Drive** (*ordinal*): The number of the drive during a game when the play occurred (indexed at one, so the first drive of the game has Drive 1 and the nth drive has Drive n)
* **qtr** (*interval*): The quarter of the game when the play occurred
* **down** (*interval*): The down when the play occurred (1st, 2nd, 3rd, or 4th)
* **TimeSecs** (*interval*): The remaining game time, in seconds, when the play began
* **yrdline100** (*ratio*): The absolute yard-line on the field where the play started (from 0 to 100, where 0 is the defensive end zone and 100 is the offensive end zone of the team with the ball)
* **ydstogo** (*ratio*): The number of yards from the line of scrimmage to the first-down line
* **ydsnet** (*ratio*): The number of yards from the beginning of the drive to the current line of scrimmage
* **GoalToGo** (*nominal*): A binary attribute whose value is 1 if there is no first down line (the end-zone is the first down line) or 0 if there is a normal first down line
* **posteam** (*nominal*): A 2-or-3 character code representing the team on offense
* **PosTeamScore** (*ratio*): The score of the team with possesion of the ball
* **DefensiveTeam** (*nominal*): A 2-or-3 character code representing the team on defense
* **ScoreDiff**: (*ratio*) The difference in score between the offensive and defensive at the time of the play.
* **PlayType**: (*nominal*) An attribute that identifies the type of play (i.e. Kickoff, Run, Pass, Sack, etc)

### Performance Metrics

The value of a classifier will be evaulated using the following cost matrix. Costs in the matrix which are set to 1 represent play predictions that would never actually occur in the context of a football game. For example, if we predicted a pass play and a kickoff occurs, then the classifier has a significant flaw. 

Bolded weights represent actual mispredictions that could occur.

|                | Actual Play | Pass | Run | Kickoff |     Punt    | Extra Point | Field Goal | Onside Kick |
|----------------|-------------|------|-----|---------|-------------|------------|-------------|-------------|
| Predicted Play |             |      |     |         |             |            |             | |
| Pass           |             | 0    | **0.1** | 1       | **0.15** | **0.15**        | **0.1**         | 1           |  
| Run            |             | **0.1**  | 0   | 1       | **0.15** | **0.15**        | **0.1**         | 1           | 
| Kickoff        |             | 1    | 1   | 0     |  1  | 1           | 1          | **0.75**       |
| Punt           |             | **0.25**    | **0.25**   | 1   | 0 |1       |  **0.15**           | 1           |
| Extra Point    |             | **0.4**  | **0.4** | 1       | 1 | 0           | 1          | 1           |
| Field Goal     |             | **0.4** | **0.4** | 1       | **0.1** | 1           | 0          | 1           |
| Onside Kick    |             | 1    | 1   | **0.25**    |  1 |1           | 1          | 0           |


This performance metric is the best for this classification problem because the actual potential cost of an incorrect play prediction varies significantly based on the nature of the misclassification. In an actual football game, it would be very costly to predict an extra point and have the opposing team run a pass play. This means that they ran a fake extra point and went for a two-point conversion. However, if a pass play is predicted and a run play occurs, the cost of the error is minimal because the defensive strategy for defending against run and pass plays.

Because the goal of this classification is to help inform defensive play-calling, a cost matrix is helpful because it allows a defensive coordinator to set his own costs to produce his own classifier, without any knowledge of the actualy computation that occurs.

### Cross Validation Methodology

We use a sequential k-fold partition of the data because this mirrors how data will be collected and analyzed. For our use, we assume that it is okay to use data in the “future” to predict data “now” because it can represent data from a previous football season. For example, if we use the first 90% of data for training and the remaining 10% of data for testing, that would simulate using most of the current season's data to predict plays towards the end of this season. If we use the first 50% and last 40% of data for training and the remaining 10% for testing, this would simulate using 40% of the previous season's data and the first 50% of this season's data to predict plays happening around the middle of the current season. 

In [4]:
from sklearn.model_selection import KFold

#Using a 10-fold sequential split.
#Note that this cv object is unused, but is here for reference
cv = KFold(n_splits=10)

y,levels = pd.factorize(df.PlayType.values)
X = df.drop('PlayType', 1).values.astype(np.float32)


num_classes = len(levels)

## Modeling

Before we build any models, we define a cost function in Python below, which is used to test all of our forthcoming models. It computes the item-wise product of a confusion matrix and our cost matrix, and returns the sum of all of the elements in the resulting matrix. We also define a function to calculate area under roc curve for a multiclass classification problem.

In [5]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc,make_scorer
from scipy import interp

import matplotlib.pyplot as plt
%matplotlib inline 
plt.style.use('ggplot')

cost_mat =     [[0  ,.1  , 1   , .15 , 0.15, .1 , 1   ],
                [.1 , 0  , 1   , 0.15, 0.15, 0.1, 1   ],
                [1  , 1  , 0   , 1   , 1   , 1  , 0.75],
                [.25,0.25, 1   , 0   , 1   ,0.15,  1  ],
                [0.4, 0.4, 1   , 1   , 0   , 1  ,  1  ],
                [0.4, 0.4, 1   , 0.1 , 1   , 0  ,  1  ],
                [1  , 1  , 0.25, 1   , 1   , 1  ,  0  ]]

def cost(Y, yhat):
    return np.sum(np.multiply(confusion_matrix(Y,yhat), cost_mat))

def auc_of_roc(Y,yhat):
    #classes = ['Pass', 'Run', 'Kickoff', 'Punt', 'Extra Point', 'Field Goal', 'Onside Kick']
    classes = range(0,7)
    
    mean_tpr = 0.0
    mean_fpr = np.linspace(0, 1, 100)
    for c in classes:
        tempY = [x==c for x in Y]
        tempYhat = [x==c for x in yhat]
        
        fpr, tpr, thresholds = roc_curve(tempY, tempYhat)
        mean_tpr += interp(mean_fpr, fpr, tpr)
        mean_tpr[0] = 0.0

        roc_auc = auc(fpr, tpr)

    mean_tpr /= len(classes)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    
    return mean_auc

auc_roc_scorer = make_scorer(auc_of_roc)
scorer = make_scorer(cost)

### Some Setup Code for TensorFlow

#### Imports

In [6]:
import tensorflow as tf
from tensorflow.contrib import learn
from tensorflow.contrib import layers

#Suppress all non-error warnings
tf.logging.set_verbosity(tf.logging.ERROR)

#### Calculating Costs for a Model

The following code performs cross validation on a model with a given step count and learning rate. This will be used as part of the grid search, as well as for evaluating the final classifier after the gridsearch is complete.

In [7]:
def get_scores_for_model(model_fn, X, y, steps=1000, learning_rate=0.05, num_splits = 10):

    auc = []
    costs = []
    
    for train_index, test_index in KFold(n_splits=num_splits).split(X, y):
        classifier = learn.TensorFlowEstimator(model_fn=model_fn, 
                                               n_classes=7, batch_size=1000,
                                               steps=steps, learning_rate=learning_rate)
        classifier.fit(X[train_index], y[train_index])
        yhat = classifier.predict(X[test_index])

        costs.append(cost(y[test_index], yhat))
        auc.append(auc_of_roc(y[test_index], yhat))

    return costs, auc

#### Grid Search

Because we're performing a grid search on a TensorFlow estimator, we use our own grid search function, instead of the one provided in sklearn, for the sake of simplicity. Our grid search will search for optimal values of *steps* and *learning_rate*. During the grid search, a subsample of 2500 items will be used, and only 3 folds of cross validation will occur. This is done to decrease computation time, which is otherwise many hours per grid search.

Note that the grid search function itself is not parallelized. This is because the underlying TensorFlow modelling is all parallelized, so maximal CPU usage is already being achieved.

In [8]:
def grid_search(model_fn, steps_list, learning_rate_list):
    
    costs = []
    
    for steps in steps_list:
        
        step_costs = []
        
        for rate in learning_rate_list:
            
            idx_sample = np.random.randint(X.shape[0], size=5000)
            step_costs.append(np.mean(get_scores_for_model(model_fn, X[idx_sample, :], y[idx_sample], steps, rate, 3)[0]))
        
        costs.append(step_costs)
        
    max_idx = np.argmax(costs)
    
    return costs, steps_list[max_idx//len(costs[0])], learning_rate_list[max_idx%len(costs[0])]

In [9]:
from matplotlib import pyplot as plt

def grid_search_heatmap(costs):
    
    plt.pcolor(np.array(costs))

SyntaxError: invalid syntax (<ipython-input-9-94829740dcd1>, line 1)

### First Deep Learning Architecture

The first deep learning architecture will be adapted from a model designed by PayPal that is used for anomaly detection. Because the vast majority of football plays are either runs or passes, and there are only a few anomalous plays, like onside kicks, etc., it stands to reason that an anomaly detection architecture would perform well for this classification task.

The architecture is quite simple: it consists of a set of 6 fully connected layers of 700 neurons, followed by a hyperbolic tangent activation function and then a single fully connected layer for output. We will adapt this model slightly to allow for the embedding of the team attributes. We will split the data into embedding and non-embedding data, run each subset of data through 6 fully connected layers of 700 neurons, and then combine their output as the input into a single final layer, used for classification. A simple drawing of the architecture is shown below.

<img src="NetworkDrawings/network1.png">

<a href="http://university.h2o.ai/cds-lp/cds02.html?mkt_tok=3RkMMJWWfF9wsRonvanAZKXonjHpfsX56%2BkqUaG0lMI%2F0ER3fOvrPUfGjI4ATsBlI%2BSLDwEYGJlv6SgFTLTBMbBrwrgKXBk%3D">
The original talk about this architecture can be found here.
</a>

##### Defining the Model

In [10]:
def deep_model_1(X, y):

    #Embeddings layer
    teamembeddings = layers.stack(X[:,11:75], layers.fully_connected, [700 for _ in range(6)])
    teamembeddings = tf.nn.tanh(teamembeddings)
    
    #Non-embeddings features
    otherfeatures = X[:,0:10]
    otherfeatures = layers.stack(otherfeatures, layers.fully_connected, [700 for _ in range(6)])
    
    tensors = tf.concat(1, [teamembeddings, otherfeatures])
    tensors = tf.nn.tanh(tensors)
    
    pred,loss = learn.models.logistic_regression(tensors, y)

    return pred, loss

##### Grid Searching on the Model

A grid search is performed on the model to find the approximately optimal step count and learning rate for the TensorFlowEstimator.

In [ ]:
costs, optimal_steps, optimal_rate = grid_search(deep_model_1, [250,500,1000,1500,2000], [.05, .01, .005, .001, .0005])
print((optimal_steps, optimal_rate))

In [ ]:
print(optimal_steps)
print(optimal_rate)

Although the grid search returned the optimal step count and rate, it is meaningful to visualize the grid that was generated, to get an idea for how much better these particular hyperparameters are than the other possible combinations in the grid.

In [ ]:
grid_search_heatmap(costs)

##### Calculating Costs and Area under ROC Curve Scores for the Model

With the optimal step count and learning rate, the costs of the model built with the given step count and rate are computed.

In [10]:
costs_model_1, auc_roc_model_1 = get_scores_for_model(deep_model_1, X, y, optimal_steps, optimal_rate)

print(costs_model_1)
print(auc_roc_model_1)

[1105.4, 1115.0, 1199.25, 1107.95, 1185.5, 1175.5, 1151.9, 1105.65, 1183.9, 1159.35]
[0.77787139, 0.79362182, 0.79940667, 0.78404317, 0.75690638, 0.79080963, 0.7568719, 0.75838605, 0.77075058, 0.78330822]


The costs and auc scores computed above are hard-coded below for later use, so that they don't need to be computed again.

In [9]:
costs_model_1 = [1105.4, 1115.0, 1199.25, 1107.95, 1185.5, 1175.5, 1151.9, 1105.65, 1183.9, 1159.35]
auc_roc_model_1 = [0.77787139, 0.79362182, 0.79940667, 0.78404317, 0.75690638, 
                   0.79080963, 0.7568719, 0.75838605, 0.77075058, 0.78330822]

### Second Deep Learning Architecture

DESCRIBE IT!

Based on https://oshearesearch.com/index.php/tag/deep-learning/

In [27]:
%%time

def deep_model_2(X, y):
    #Embeddings layer
    teamembeddings = layers.stack(X[:,11:75], layers.fully_connected, [20,4])
    teamembeddings = tf.nn.relu(teamembeddings)
    
    #Non-embeddings features
    otherfeatures = X[:,0:10]

    #Concatenate the embeddings with the non-embeddings
    tensors = tf.concat(1, [teamembeddings, otherfeatures])

    tensors = layers.stack(tensors, layers.fully_connected, [100,50,25, 100])
    
    #try different activation functions sigmoid
    tensors = tf.nn.relu(tensors)
    tensors = tf.nn.relu(tensors)
    
    tensors = tf.nn.softmax(tensors)

    pred, loss = learn.models.logistic_regression(tensors, y)
    
    return pred, loss
    
scores_model_2 = get_scores_for_model(deep_model_2,X,y)

[2042.75]
[0.49999999999999994]
[2042.75, 2007.5]
[0.49999999999999994, 0.49999999999999994]
[2042.75, 2007.5, 2008.75]
[0.49999999999999994, 0.49999999999999994, 0.50008261353449202]
[2042.75, 2007.5, 2008.75, 1978.0]
[0.49999999999999994, 0.49999999999999994, 0.50008261353449202, 0.49999999999999994]
[2042.75, 2007.5, 2008.75, 1978.0, 2059.5500000000002]
[0.49999999999999994, 0.49999999999999994, 0.50008261353449202, 0.49999999999999994, 0.50000496933088179]
[2042.75, 2007.5, 2008.75, 1978.0, 2059.5500000000002, 2021.75]
[0.49999999999999994, 0.49999999999999994, 0.50008261353449202, 0.49999999999999994, 0.50000496933088179, 0.49999999999999994]
[2042.75, 2007.5, 2008.75, 1978.0, 2059.5500000000002, 2021.75, 1974.0]
[0.49999999999999994, 0.49999999999999994, 0.50008261353449202, 0.49999999999999994, 0.50000496933088179, 0.49999999999999994, 0.49999999999999994]
[2042.75, 2007.5, 2008.75, 1978.0, 2059.5500000000002, 2021.75, 1974.0, 2048.25]
[0.49999999999999994, 0.49999999999999994, 

In [28]:
print(scores_model_2)

([2042.75, 2007.5, 2008.75, 1978.0, 2059.5500000000002, 2021.75, 1974.0, 2048.25, 2027.0, 2041.25], [0.49999999999999994, 0.49999999999999994, 0.50008261353449202, 0.49999999999999994, 0.50000496933088179, 0.49999999999999994, 0.49999999999999994, 0.49999999999999994, 0.49999999999999994, 0.49999999999999994])


### Third Deep Learning Architecture

Explain it!

In [47]:
%%time

def deep_model_3(X, y):

    #Embeddings layer
    teamembeddings = layers.stack(X[:,11:75], layers.fully_connected, [20,4])
    #teamembeddings = tf.nn.relu(teamembeddings)
    
    #Non-embeddings features
    otherfeatures = X[:,0:10]

    #Concatenate the embeddings with the non-embeddings
    tensors = tf.concat(1, [teamembeddings, otherfeatures])
    
    #[500,200,100,500][1000,1000,1000,500,200,1000]
    tensors = layers.stack(tensors, layers.fully_connected, [1000,1000,1000,500,200,1000])
    
    #try different activation functions sigmoid
    tensors = tf.nn.relu(tensors)
    tensors = tf.nn.relu(tensors)
    tensors = tf.nn.relu(tensors)

    pred, loss = learn.models.logistic_regression(tensors, y)
    
    return pred, loss


optimal_steps, optimal_rate = grid_search(deep_model_3, [100,250,500], [0.005, .0005, .00005])
print("optimal steps: ", optimal_steps, " optimal rate: ", optimal_rate)
    


ValueError: operands could not be broadcast together with shapes (6,6) (7,7) 

In [50]:
#optimal steps250  optimal rate:  5e-05
#optimal_steps = 250
#optimal_rate = .00005
scores_model_3 = get_scores_for_model(deep_model_3,X,y)#,optimal_steps,optimal_rate)
print(scores_model_3)

([1654.9499999999998, 1546.3000000000002, 1583.3000000000002, 1547.2, 1640.95, 1634.7, 1665.25, 1626.7, 1678.3, 1639.5500000000002], [0.60113696461703936, 0.62256431559564995, 0.59913643227078495, 0.61860384436436899, 0.5933883723124902, 0.64275680858409689, 0.58600068196046851, 0.62421285285956252, 0.5873372252774246, 0.5973412561138246])


### Architecture Comparison

The following getDifference function is used to create a tuple of the range of possible differences of mean for two sets of scores (cost or auc_roc), with 95% confidence. We use the second of the two confidence interval tests proposed in the ICA3 reversed assignment, because the datasets cannot be assumed to be independent, so the binomial approximation to the normal distribution does not hold.

In [ ]:
def getDifference(cost1,cost2,z_val=2.26,size=10):
    diff12 = cost1 - cost2
    sigma12 = np.sqrt(np.sum(diff12*diff12) * 1/(size-1))
    d12 = (np.mean(diff12) + 1/(np.sqrt(size)) * z_val * sigma12, np.mean(diff12) - 1/(np.sqrt(size)) * z_val * sigma12)
    return d12

#### Cost Difference

The getDifference function is now used to create confidence intervals for the cost differences of the 3 possible pairs of two architectures created from the set of 3 architectures.

In [ ]:
d_one_two = np.array(getDifference(costs_model_1, costs_model_2))
d_one_three = np.array(getDifference(costs_model_1, costs_model_3))
d_two_three = np.array(getDifference(costs_model_2, costs_model_3))

print("Average Model 1 vs Model 2 Difference:", d_one_two)
print("Average Model 1 vs Model 3 Difference:", d_one_three)
print("Average Model 2 vs Model 3 Difference:", d_two_three)

THIS SHOWS THAT... EXPLAIN

#### Area Under Multiclass ROC Curve Difference

The same process is now used for confidence intervals for the auc_roc metric.

In [ ]:
d_one_two = np.array(getDifference(auc_roc_model_1, auc_roc_model_2))
d_one_three = np.array(getDifference(auc_roc_model_1, auc_roc_model_3))
d_two_three = np.array(getDifference(auc_roc_model_2, auc_roc_model_3))

print("Average Model 1 vs Model 2 Difference:", d_one_two)
print("Average Model 1 vs Model 3 Difference:", d_one_three)
print("Average Model 2 vs Model 3 Difference:", d_two_three)

THIS SHOWS THAT... EXPLAIN

## Deployment

HOW ARE WE GOING TO DEPLOY? CAN WE EVEN BEAT RANDOM FORESTS?